In [ ]:
import os 
os.chdir("../")

In [ ]:
%pwd

## ini langkah langkah jika lupa versi dan otomatis terupgrade dan ini merupakan versi paling stabil dari projekku ##

In [ ]:
pip install langchain==0.2.11

In [ ]:
pip install langchain_huggingface

In [ ]:
pip install langchain_pinecone

In [ ]:
import json
from langchain_core.documents import Document
import re

def load_and_prepare_docs_from_json(json_path: str) -> list[Document]:
    """
    Memuat data wisata dari file JSON dan mengubahnya menjadi format
    LangChain Document, siap untuk dimasukkan ke Pinecone.
    """
    
    print(f"Memuat data dari {json_path}...")
    
    # Fungsi helper sederhana untuk membuat ID unik
    
    all_docs = []
    
    # Muat file JSON
    with open(json_path, 'r', encoding='utf-8') as f:
        data_wisata = json.load(f) #

    for item in data_wisata:
        # 1. Gabungkan semua informasi penting menjadi 'page_content'
        # Ini akan menjadi teks yang di-embed oleh model.
        content = f"""
        Nama Wisata: {item['nama']}
        Lokasi: {item['lokasi']}
        Deskripsi: {item['deskripsi']}
        Aktivitas Utama: {item.get('aktivitas_utama', 'Tidak ada info')}
        Harga Tiket: Dewasa {item['harga_tiket_dewasa']}, Anak {item['harga_tiket_anak_anak']}, Weekend {item['harga_tiket_weekend']}
        Jam Operasional: {item['jam_buka']} sampai {item['jam_tutup']}
        """

        metadata = {
            "id": item['id'],
            "nama": item['nama'],
            "lokasi": item['lokasi'],
            "sumber": json_path
        }

        
        doc = Document(page_content=content, metadata=metadata)
        all_docs.append(doc)
    
    print(f"Berhasil memuat dan mengubah {len(all_docs)} dokumen.")
    return all_docs

JSON_FILE_PATH = "data/data-wisata.json" 
all_docs = load_and_prepare_docs_from_json(JSON_FILE_PATH)

print("\n--- Contoh Dokumen Pertama ---")
print(all_docs[0].page_content)
print("\n--- Contoh Metadata Pertama ---")
print(all_docs[0].metadata)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

def download_embeddings():
    model_name = "intfloat/multilingual-e5-large"
    return HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
    )

embeddings = download_embeddings()
print("OK:", len(embeddings.embed_query("tes")))

In [ ]:
embeddings

In [ ]:
daftar_teks = ["Hallo dunia"]
vectors = embeddings.embed_documents(daftar_teks)

print(vectors)

In [ ]:
print(f"Vector length: {len(vectors[0])}")

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

#LOAD API 

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not PINECONE_API_KEY or (not OPENAI_API_KEY ):
    print("Error: Pastikan API key sudah diatur di file .env")
else:
    print("API keys berhasil dimuat!")

In [ ]:
pip install pinecone

In [10]:
import pinecone
print("Pinecone version:", pinecone.__version__)


DeprecatedPluginError: The `pinecone-plugin-inference` package has been deprecated. The features from that plugin have been incorporated into the main `pinecone` package with no need for additional plugins. Please remove the `pinecone-plugin-inference` package from your dependencies to ensure you have the most up-to-date version of these features.

In [ ]:
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = PINECONE_API_KEY

pc= Pinecone(api_key=pinecone_api_key)

In [ ]:
pc

In [ ]:
from pinecone import ServerlessSpec 

index_name = "chatbot-wisata-e5large"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=1024,  
        metric= "cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [ ]:
import pinecone
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=all_docs,  # Gunakan variabel all_docs dari JSON
    embedding=embeddings,
    index_name=index_name
)

print("Upload ke Pinecone berhasil.")

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":1})
# retriever = docsearch.as_retriever(
#     search_type="mmr",
#     search_kwargs={"k": 10, "fetch_k": 100, "lambda_mult": 0.5}
# )

In [ ]:
from langchain_core.globals import set_debug
set_debug(True)
retrieved_docs = retriever.invoke("harga tiket pancuran pitu ?")
retrieved_docs

In [ ]:
retrieved_docs = retriever.invoke("harga tiket anak-anak pancuran pitu ?")
retrieved_docs

## TEST MODEL LAIN BISA DISINI gpt 4.o Mini## 

In [ ]:
from langchain_core.globals import set_verbose
set_verbose(True)

from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4o-mini",temperature=0.1)


#CEK DULU LONGCHAINNYA

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# system_prompt = (
#   "Anda asisten pariwisata (Bahasa Indonesia). Anda adalah sistem penjawab. Jawab pertanyaan HANYA menggunakan fakta dari konteks. JANGAN TAMBAHKAN kata-kata pembuka atau penutup.{context}"
#   "dengan mengutamakan kata/frasa PERSIS dari konteks. Hindari sinonim jika tidak perlu."
#   "Jika tidak tahu, jawab 'Saya tidak menemukan informasinya.' "
#   "Gunakan maksimal 3 kalimat."
#   "\n\n"
#   "{context}"
# )


# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt),
#         ("human", "{input}"),
#     ]
# )

In [ ]:
system_prompt = (
   "Anda adalah asisten pariwisata Banyumas yang menjawab dalam Bahasa Indonesia.\n\n"
    
    "INSTRUKSI:\n"
    "• Jawab pertanyaan HANYA menggunakan fakta dari konteks di bawah\n"
    "• Gunakan kata-kata yang PERSIS dari konteks untuk menjaga akurasi\n"
    "• Berikan jawaban langsung tanpa kata pembuka seperti 'Berdasarkan konteks...'\n"
    "• Jika informasi tidak tersedia di konteks, jawab: 'Saya tidak menemukan informasi tersebut.'\n"
    "• Susun jawaban secara ringkas (maksimal 3-4 kalimat) namun informatif\n\n"
    
    "KONTEKS:\n{context}\n"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "berapa harga tiket Dreamland?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "rekomendasi wisata alam di banyumas ?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": " rekomendasi wisata di banyumas selain wisata alam?"})
print(response["answer"])


In [ ]:
response = rag_chain.invoke({"input": " aktivitas apa yang bisa dilakukan di pancuran pitu?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": " aktivitas apa yang bisa dilakukan di baturraden?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "Goa Maria Kaliori itu apa?"})
print(response["answer"])

## TEST PERTANYAAAN MODEL gpt-4.o-mini##


In [ ]:
response = rag_chain.invoke({"input":"pagubugan melung itu apa?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input":"harga tiket Lembu Benggolo farm Resort dan jam bukanya?"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input":" aktivitas apa yang bisa dilakukan di baturraden?"})
print(response["answer"])

## SELESAI TEST PERTANYAAAN ##

##  TEST SKOR BLEU Latensi Rouge dan F1 Skor MODEL ##
## gpt-4o-mini ##

## FINAL KODE
## PENGUJIAN GENERATIVE dan Retreival Single question 

In [ ]:
import time
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from typing import List, Dict
from bert_score import score as bert_scorer
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

test_set =  [
  {
    "question": "Di mana lokasi Lokawisata Baturraden?",
    "true_context_id": "baturraden",
    "true_answer": "Di Karangmangu, Baturraden."
  },
  {
    "question": "Aktivitas di Lokawisata Baturraden?",
    "true_context_id": "baturraden",
    "true_answer": "Rekreasi keluarga, Pemandian air panas, dan Taman bermain."
  },
  {
    "question": "Tiket dewasa Lokawisata Baturraden (weekday)?",
    "true_context_id": "baturraden",
    "true_answer": "Rp20.000."
  },
  {
    "question": "Tiket anak Lokawisata Baturraden (weekday)?",
    "true_context_id": "baturraden",
    "true_answer": "Rp10.000."
  },
  {
    "question": "Tiket dewasa Lokawisata Baturraden (weekend)?",
    "true_context_id": "baturraden",
    "true_answer": "Rp25.000."
  },
  {
    "question": "Tiket anak Lokawisata Baturraden (weekend)?",
    "true_context_id": "baturraden",
    "true_answer": "Rp12.500."
  },
  {
    "question": "Jam operasional Lokawisata Baturraden?",
    "true_context_id": "baturraden",
    "true_answer": "Buka 7 AM dan tutup 7 PM."
  },
  {
    "question": "Di mana lokasi Telaga Sunyi?",
    "true_context_id": "telaga_sunyi",
    "true_answer": "Di Ketenger, Baturraden."
  },
  {
    "question": "Suasana di Telaga Sunyi?",
    "true_context_id": "telaga_sunyi",
    "true_answer": "Tenang dan sejuk, cocok untuk bersantai."
  },
  {
    "question": "Berapa tiket dewasa Telaga Sunyi?",
    "true_context_id": "telaga_sunyi",
    "true_answer": "Rp15.000."
  },
  {
    "question": "Jam operasional Telaga Sunyi?",
    "true_context_id": "telaga_sunyi",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Hutan Pinus Limpakuwus?",
    "true_context_id": "limpakuwus",
    "true_answer": "Di Jl Baturraden Timur, Limpakuwus, Sumbang."
  },
  {
    "question": "Aktivitas di Hutan Pinus Limpakuwus?",
    "true_context_id": "limpakuwus",
    "true_answer": "Camping, Fotografi, dan Wahana permainan."
  },
  {
    "question": "Berapa tiket dewasa Hutan Pinus Limpakuwus?",
    "true_context_id": "limpakuwus",
    "true_answer": "Rp17.500."
  },
  {
    "question": "Jam operasional Hutan Pinus Limpakuwus?",
    "true_context_id": "limpakuwus",
    "true_answer": "Buka 7.30 AM dan tutup 4.30 PM."
  },
  {
    "question": "Di mana lokasi Curug Jenggala?",
    "true_context_id": "curug_jenggala",
    "true_answer": "Di Kalipagu, Ketenger."
  },
  {
    "question": "Apa keunikan Curug Jenggala?",
    "true_context_id": "curug_jenggala",
    "true_answer": "Tiga aliran air terjun dan dek berbentuk hati yang ikonik."
  },
  {
    "question": "Berapa tiket dewasa Curug Jenggala?",
    "true_context_id": "curug_jenggala",
    "true_answer": "Rp15.000."
  },
  {
    "question": "Jam operasional Curug Jenggala?",
    "true_context_id": "curug_jenggala",
    "true_answer": "Buka 7.30 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Curug Telu?",
    "true_context_id": "curug_telu",
    "true_answer": "Di Karangsalam, Baturraden."
  },
  {
    "question": "Apa deskripsi Curug Telu?",
    "true_context_id": "curug_telu",
    "true_answer": "Memiliki tiga air terjun berurutan."
  },
  {
    "question": "Berapa tiket dewasa Curug Telu?",
    "true_context_id": "curug_telu",
    "true_answer": "Rp5.000."
  },
  {
    "question": "Jam operasional Curug Telu?",
    "true_context_id": "curug_telu",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Curug Bayan?",
    "true_context_id": "curug_bayan",
    "true_answer": "Di Ketenger, Baturraden."
  },
  {
    "question": "Deskripsi Curug Bayan?",
    "true_context_id": "curug_bayan",
    "true_answer": "Pemandangan air terjun bertingkat di suasana pedesaan."
  },
  {
    "question": "Berapa tiket dewasa Curug Bayan?",
    "true_context_id": "curug_bayan",
    "true_answer": "Rp15.000."
  },
  {
    "question": "Jam operasional Curug Bayan?",
    "true_context_id": "curug_bayan",
    "true_answer": "Buka 8 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Curug Cipendok?",
    "true_context_id": "curug_cipendok",
    "true_answer": "Di Karangtengah, Cilongok."
  },
  {
    "question": "Berapa tinggi Curug Cipendok?",
    "true_context_id": "curug_cipendok",
    "true_answer": "Sekitar 90 meter."
  },
  {
    "question": "Berapa tiket dewasa Curug Cipendok?",
    "true_context_id": "curug_cipendok",
    "true_answer": "Rp15.000."
  },
  {
    "question": "Berapa tiket anak Curug Cipendok?",
    "true_context_id": "curug_cipendok",
    "true_answer": "5000."
  },
  {
    "question": "Berapa tiket weekend Curug Cipendok?",
    "true_context_id": "curug_cipendok",
    "true_answer": "Rp12.500 (Libur besar)."
  },
  {
    "question": "Jam operasional Curug Cipendok?",
    "true_context_id": "curug_cipendok",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Curug Gomblang?",
    "true_context_id": "curug_gomblang",
    "true_answer": "Di Kalisalak, Kedungbanteng."
  },
  {
    "question": "Fasilitas di Curug Gomblang?",
    "true_context_id": "curug_gomblang",
    "true_answer": "Camping ground dan spot swafoto."
  },
  {
    "question": "Berapa tiket dewasa Curug Gomblang?",
    "true_context_id": "curug_gomblang",
    "true_answer": "Rp10.000."
  },
  {
    "question": "Jam operasional Curug Gomblang?",
    "true_context_id": "curug_gomblang",
    "true_answer": "Buka 8 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Curug Ceheng?",
    "true_context_id": "curug_ceheng",
    "true_answer": "Di Gandatapa, Sumbang."
  },
  {
    "question": "Aktivitas di Curug Ceheng?",
    "true_context_id": "curug_ceheng",
    "true_answer": "Rekreasi alam dan berkemah ringan."
  },
  {
    "question": "Berapa tiket dewasa Curug Ceheng?",
    "true_context_id": "curug_ceheng",
    "true_answer": "Rp5.000."
  },
  {
    "question": "Jam operasional Curug Ceheng?",
    "true_context_id": "curug_ceheng",
    "true_answer": "Buka 8 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Curug Nangga?",
    "true_context_id": "curug_nangga",
    "true_answer": "Di Petahunan, Pekuncen."
  },
  {
    "question": "Apa keunikan Curug Nangga?",
    "true_context_id": "curug_nangga",
    "true_answer": "Terkenal dengan tujuh tingkat air terjun."
  },
  {
    "question": "Berapa tiket dewasa Curug Nangga?",
    "true_context_id": "curug_nangga",
    "true_answer": "Rp10.000."
  },
  {
    "question": "Jam operasional Curug Nangga?",
    "true_context_id": "curug_nangga",
    "true_answer": "Buka 8 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Bukit Tranggulasih?",
    "true_context_id": "bukit_tranggulasih",
    "true_answer": "Di Windujaya, Kedungbanteng."
  },
  {
    "question": "Pemandangan di Bukit Tranggulasih?",
    "true_context_id": "bukit_tranggulasih",
    "true_answer": "Pemandangan Kota Purwokerto dari ketinggian."
  },
  {
    "question": "Aktivitas di Bukit Tranggulasih?",
    "true_context_id": "bukit_tranggulasih",
    "true_answer": "Camping dan berburu sunrise."
  },
  {
    "question": "Berapa tiket dewasa Bukit Tranggulasih?",
    "true_context_id": "bukit_tranggulasih",
    "true_answer": "Rp5.000."
  },
  {
    "question": "Jam operasional Bukit Tranggulasih?",
    "true_context_id": "bukit_tranggulasih",
    "true_answer": "24 Hours."
  },
  {
    "question": "Di mana lokasi Bukit Watu Meja?",
    "true_context_id": "bukit_watu_meja",
    "true_answer": "Di Tumiyang, Kebasen."
  },
  {
    "question": "Pemandangan di Bukit Watu Meja?",
    "true_context_id": "bukit_watu_meja",
    "true_answer": "Panorama Sungai Serayu dan hamparan pegunungan."
  },
  {
    "question": "Berapa tiket dewasa Bukit Watu Meja?",
    "true_context_id": "bukit_watu_meja",
    "true_answer": "Rp7.000."
  },
  {
    "question": "Jam operasional Bukit Watu Meja?",
    "true_context_id": "bukit_watu_meja",
    "true_answer": "Buka 8 AM dan tutup 6 PM."
  },
  {
    "question": "Di mana lokasi Kebun Raya Baturraden?",
    "true_context_id": "kebun_raya_btr",
    "true_answer": "Di Kemutug lor, Baturraden."
  },
  {
    "question": "Deskripsi Kebun Raya Baturraden?",
    "true_context_id": "kebun_raya_btr",
    "true_answer": "Taman botani dengan koleksi flora pegunungan dan rumah kaca."
  },
  {
    "question": "Berapa tiket dewasa Kebun Raya Baturraden (weekday)?",
    "true_context_id": "kebun_raya_btr",
    "true_answer": "Rp20.000."
  },
  {
    "question": "Berapa tiket weekend Kebun Raya Baturraden?",
    "true_context_id": "kebun_raya_btr",
    "true_answer": "Rp25.000."
  },
  {
    "question": "Jam operasional Kebun Raya Baturraden?",
    "true_context_id": "kebun_raya_btr",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Pancuran Telu?",
    "true_context_id": "pancuran_telu",
    "true_answer": "Di Karangmangu, Baturraden."
  },
  {
    "question": "Deskripsi Pancuran Telu?",
    "true_context_id": "pancuran_telu",
    "true_answer": "Pemandian air panas alami (air belerang)."
  },
  {
    "question": "Berapa tiket dewasa Pancuran Telu (weekday)?",
    "true_context_id": "pancuran_telu",
    "true_answer": "Rp20.000."
  },
  {
    "question": "Berapa tiket weekend Pancuran Telu?",
    "true_context_id": "pancuran_telu",
    "true_answer": "Rp25.000."
  },
  {
    "question": "Jam operasional Pancuran Telu?",
    "true_context_id": "pancuran_telu",
    "true_answer": "Buka 7 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Pancuran 7?",
    "true_context_id": "pancuran_pitu",
    "true_answer": "Di Ketenger, Baturraden."
  },
  {
    "question": "Apa keunikan Pancuran 7?",
    "true_context_id": "pancuran_pitu",
    "true_answer": "Pemandian air panas dengan tujuh pancuran alami."
  },
  {
    "question": "Berapa tiket dewasa Pancuran 7?",
    "true_context_id": "pancuran_pitu",
    "true_answer": "Rp15.000."
  },
  {
    "question": "Jam operasional Pancuran 7?",
    "true_context_id": "pancuran_pitu",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Baturraden Adventure Forest?",
    "true_context_id": "baf",
    "true_answer": "Di Pandak, Karangsalam, Baturraden."
  },
  {
    "question": "Aktivitas di Baturraden Adventure Forest?",
    "true_context_id": "baf",
    "true_answer": "Outbound, Flying fox, dan Camping."
  },
  {
    "question": "Berapa tiket Baturraden Adventure Forest?",
    "true_context_id": "baf",
    "true_answer": "Tidak Ada."
  },
  {
    "question": "Jam operasional Baturraden Adventure Forest?",
    "true_context_id": "baf",
    "true_answer": "Buka 8 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Wana Wisata Baturraden?",
    "true_context_id": "wana_baturraden",
    "true_answer": "Di Baturraden."
  },
  {
    "question": "Aktivitas di Wana Wisata Baturraden?",
    "true_context_id": "wana_baturraden",
    "true_answer": "Trekking dan Camping (Bumi Perkemahan)."
  },
  {
    "question": "Berapa tiket Wana Wisata Baturraden?",
    "true_context_id": "wana_baturraden",
    "true_answer": "Tidak Ada."
  },
  {
    "question": "Jam operasional Wana Wisata Baturraden?",
    "true_context_id": "wana_baturraden",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Bhumi Bambu Baturraden?",
    "true_context_id": "bhumi_bambu",
    "true_answer": "Di Baturraden."
  },
  {
    "question": "Apa keunikan Bhumi Bambu Baturraden?",
    "true_context_id": "bhumi_bambu",
    "true_answer": "Hutan bambu dengan empat air terjun alami."
  },
  {
    "question": "Berapa tiket Bhumi Bambu Baturraden?",
    "true_context_id": "bhumi_bambu",
    "true_answer": "Rp25.000."
  },
  {
    "question": "Jam operasional Bhumi Bambu Baturraden?",
    "true_context_id": "bhumi_bambu",
    "true_answer": "Buka 8 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Bukit Pandang Munggang?",
    "true_context_id": "bukit_pandang",
    "true_answer": "Di Ketenger, Baturraden."
  },
  {
    "question": "Pemandangan di Bukit Pandang Munggang?",
    "true_context_id": "bukit_pandang",
    "true_answer": "Panorama Kota Purwokerto dari ketinggian."
  },
  {
    "question": "Berapa tiket Bukit Pandang Munggang?",
    "true_context_id": "bukit_pandang",
    "true_answer": "Tidak Ada."
  },
  {
    "question": "Jam operasional Bukit Pandang Munggang?",
    "true_context_id": "bukit_pandang",
    "true_answer": "Buka 6 AM dan tutup 6 PM."
  },
  {
    "question": "Di mana lokasi Curug Kedung Lesung?",
    "true_context_id": "curug_kedung_lesung",
    "true_answer": "Di Tumiyang, Pekuncen."
  },
  {
    "question": "Deskripsi Curug Kedung Lesung?",
    "true_context_id": "curug_kedung_lesung",
    "true_answer": "Air terjun kecil dengan kolam alami yang jernih."
  },
  {
    "question": "Berapa tiket dewasa Curug Kedung Lesung?",
    "true_context_id": "curug_kedung_lesung",
    "true_answer": "Rp5.000."
  },
  {
    "question": "Jam operasional Curug Kedung Lesung?",
    "true_context_id": "curug_kedung_lesung",
    "true_answer": "Buka 8 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Curug Grinjing?",
    "true_context_id": "curug_grinjing",
    "true_answer": "Di Banyumas."
  },
  {
    "question": "Suasana di Curug Grinjing?",
    "true_context_id": "curug_grinjing",
    "true_answer": "Sunyi dan alami (air terjun tersembunyi)."
  },
  {
    "question": "Berapa tiket Curug Grinjing?",
    "true_context_id": "curug_grinjing",
    "true_answer": "Tidak Ada."
  },
  {
    "question": "Jam operasional Curug Grinjing?",
    "true_context_id": "curug_grinjing",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Curug Tirta Sela?",
    "true_context_id": "curug_tirta_sela",
    "true_answer": "Di Banyumas."
  },
  {
    "question": "Deskripsi Curug Tirta Sela?",
    "true_context_id": "curug_tirta_sela",
    "true_answer": "Lokasi tersembunyi di desa wisata, air jernih."
  },
  {
    "question": "Berapa tiket Curug Tirta Sela?",
    "true_context_id": "curug_tirta_sela",
    "true_answer": "Tidak Ada."
  },
  {
    "question": "Jam operasional Curug Tirta Sela?",
    "true_context_id": "curug_tirta_sela",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Curug Gemawang?",
    "true_context_id": "curug_gemawang",
    "true_answer": "Di Banyumas."
  },
  {
    "question": "Apa keunikan Curug Gemawang?",
    "true_context_id": "curug_gemawang",
    "true_answer": "Air terjun dengan kolam biru tosca."
  },
  {
    "question": "Berapa tiket dewasa Curug Gemawang?",
    "true_context_id": "curug_gemawang",
    "true_answer": "Rp5.000."
  },
  {
    "question": "Jam operasional Curug Gemawang?",
    "true_context_id": "curug_gemawang",
    "true_answer": "Buka 7 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Pagubugan Melung?",
    "true_context_id": "pagubugan_melung",
    "true_answer": "Di Melung, Banyumas."
  },
  {
    "question": "Jarak Pagubugan Melung dari pusat Banyumas?",
    "true_context_id": "pagubugan_melung",
    "true_answer": "Kurang lebih 13 km."
  },
  {
    "question": "Berapa tiket dewasa Pagubugan Melung?",
    "true_context_id": "pagubugan_melung",
    "true_answer": "Rp10.000."
  },
  {
    "question": "Jam operasional Pagubugan Melung?",
    "true_context_id": "pagubugan_melung",
    "true_answer": "Buka 7.30 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Taman Panemon?",
    "true_context_id": "taman_panemon",
    "true_answer": "Di Kebanggan, Sumbang Banyumas."
  },
  {
    "question": "Apa konsep Taman Panemon?",
    "true_context_id": "taman_panemon",
    "true_answer": "Kolam renang dengan konsep pantai."
  },
  {
    "question": "Berapa tiket dewasa Taman Panemon (weekday)?",
    "true_context_id": "taman_panemon",
    "true_answer": "Rp15.000."
  },
  {
    "question": "Berapa tiket weekend Taman Panemon?",
    "true_context_id": "taman_panemon",
    "true_answer": "Rp20.000."
  },
  {
    "question": "Jam operasional Taman Panemon?",
    "true_context_id": "taman_panemon",
    "true_answer": "Buka 8 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Taman Budaya Soetedja?",
    "true_context_id": "taman_budaya_soetedja",
    "true_answer": "Di Purwokerto Kidul, Banyumas."
  },
  {
    "question": "Apa fungsi Taman Budaya Soetedja?",
    "true_context_id": "taman_budaya_soetedja",
    "true_answer": "Tempat destinasi seni dan budaya (pertunjukan kesenian)."
  },
  {
    "question": "Berapa tiket Taman Budaya Soetedja?",
    "true_context_id": "taman_budaya_soetedja",
    "true_answer": "Gratis."
  },
  {
    "question": "Jam operasional Taman Budaya Soetedja?",
    "true_context_id": "taman_budaya_soetedja",
    "true_answer": "Buka 7 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Museum Jendral Soedirman?",
    "true_context_id": "museum_jensoed",
    "true_answer": "Di Pasir Kidul Banyumas."
  },
  {
    "question": "Aktivitas di Museum Jendral Soedirman?",
    "true_context_id": "museum_jensoed",
    "true_answer": "Wisata edukasi, Belajar sejarah, Melihat diorama."
  },
  {
    "question": "Berapa tiket Museum Jendral Soedirman?",
    "true_context_id": "museum_jensoed",
    "true_answer": "Rp3.000."
  },
  {
    "question": "Jam operasional Museum Jendral Soedirman?",
    "true_context_id": "museum_jensoed",
    "true_answer": "Buka 7 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Situs Watu Lumpang?",
    "true_context_id": "situs_watu_lumpang",
    "true_answer": "Di Cilongok Banyumas."
  },
  {
    "question": "Deskripsi Situs Watu Lumpang?",
    "true_context_id": "situs_watu_lumpang",
    "true_answer": "Peninggalan purbakala zaman Megalitikum."
  },
  {
    "question": "Berapa tiket Situs Watu Lumpang?",
    "true_context_id": "situs_watu_lumpang",
    "true_answer": "Gratis."
  },
  {
    "question": "Jam operasional Situs Watu Lumpang?",
    "true_context_id": "situs_watu_lumpang",
    "true_answer": "24 Hours."
  },
  {
    "question": "Di mana lokasi Makam R. Djoko Kahiman?",
    "true_context_id": "makam_djoko_kahiman",
    "true_answer": "Di Dawuhan, Kecamatan Banyumas."
  },
  {
    "question": "Aktivitas di Makam R. Djoko Kahiman?",
    "true_context_id": "makam_djoko_kahiman",
    "true_answer": "Wisata religi dan Ziarah makam."
  },
  {
    "question": "Berapa tiket Makam R. Djoko Kahiman?",
    "true_context_id": "makam_djoko_kahiman",
    "true_answer": "Gratis."
  },
  {
    "question": "Jam operasional Makam R. Djoko Kahiman?",
    "true_context_id": "makam_djoko_kahiman",
    "true_answer": "24 Hours."
  },
  {
    "question": "Di mana lokasi Museum Wayang Banyumas?",
    "true_context_id": "museum_wayang",
    "true_answer": "Di Sudagaran, Kec. Banyumas."
  },
  {
    "question": "Apa saja koleksi Museum Wayang Banyumas?",
    "true_context_id": "museum_wayang",
    "true_answer": "Wayang Gagrag Banyumasan, Gamelan, Calung, Tosan Aji."
  },
  {
    "question": "Berapa tiket Museum Wayang Banyumas?",
    "true_context_id": "museum_wayang",
    "true_answer": "Rp1.000."
  },
  {
    "question": "Jam operasional Museum Wayang Banyumas?",
    "true_context_id": "museum_wayang",
    "true_answer": "Buka 9 AM dan tutup 3 PM."
  },
  {
    "question": "Di mana lokasi Taman Tirta Alami Pancasan?",
    "true_context_id": "taman_tirta_alami_pancasan",
    "true_answer": "Di Pancasan, Kec. Ajibarang."
  },
  {
    "question": "Dari mana sumber air Taman Tirta Alami Pancasan?",
    "true_context_id": "taman_tirta_alami_pancasan",
    "true_answer": "Berasal dari Sungai Tajum."
  },
  {
    "question": "Berapa tiket dewasa Taman Tirta Alami Pancasan?",
    "true_context_id": "taman_tirta_alami_pancasan",
    "true_answer": "8000."
  },
  {
    "question": "Berapa tiket weekend Taman Tirta Alami Pancasan?",
    "true_context_id": "taman_tirta_alami_pancasan",
    "true_answer": "Rp10.000."
  },
  {
    "question": "Jam operasional Taman Tirta Alami Pancasan?",
    "true_context_id": "taman_tirta_alami_pancasan",
    "true_answer": "Buka 7 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Kidung Kampoengku?",
    "true_context_id": "kidung_kampoengku",
    "true_answer": "Di Karangnangka, Kec. Kedungbanteng."
  },
  {
    "question": "Wahana di Kidung Kampoengku?",
    "true_context_id": "kidung_kampoengku",
    "true_answer": "Outbound, River Tubing, Flying Fox, dan Kolam Renang."
  },
  {
    "question": "Berapa tiket Kidung Kampoengku?",
    "true_context_id": "kidung_kampoengku",
    "true_answer": "Rp15.000."
  },
  {
    "question": "Jam operasional Kidung Kampoengku?",
    "true_context_id": "kidung_kampoengku",
    "true_answer": "Buka 6 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Depo Bay?",
    "true_context_id": "depo_bay",
    "true_answer": "Di Kalikidang, Kecamatan Sokaraja."
  },
  {
    "question": "Wahana di Depo Bay?",
    "true_context_id": "depo_bay",
    "true_answer": "Waterpark, Kolam ombak, Seluncuran, dan Ember tumpah."
  },
  {
    "question": "Berapa tiket Depo Bay (weekday)?",
    "true_context_id": "depo_bay",
    "true_answer": "Rp20.000."
  },
  {
    "question": "Berapa tiket Depo Bay (weekend)?",
    "true_context_id": "depo_bay",
    "true_answer": "Rp30.000."
  },
  {
    "question": "Jam operasional Depo Bay?",
    "true_context_id": "depo_bay",
    "true_answer": "Buka 9 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Wisata Outbound Candi?",
    "true_context_id": "outbound_candi",
    "true_answer": "Di Karangkemiri, Karanglewas."
  },
  {
    "question": "Aktivitas di Wisata Outbound Candi?",
    "true_context_id": "outbound_candi",
    "true_answer": "Outbound, Wisata edukasi (Perkebunan), dan Berenang."
  },
  {
    "question": "Berapa tiket Wisata Outbound Candi?",
    "true_context_id": "outbound_candi",
    "true_answer": "Rp15.000."
  },
  {
    "question": "Jam operasional Wisata Outbound Candi?",
    "true_context_id": "outbound_candi",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi The Forest Island?",
    "true_context_id": "the_forest_island",
    "true_answer": "Di Pandak, Baturraden."
  },
  {
    "question": "Fasilitas unik The Forest Island?",
    "true_context_id": "the_forest_island",
    "true_answer": "Kolam indoor khusus wanita dan taman bunga."
  },
  {
    "question": "Berapa tiket The Forest Island?",
    "true_context_id": "the_forest_island",
    "true_answer": "Rp 10.000 - Rp 20.000."
  },
  {
    "question": "Hari apa The Forest Island tutup?",
    "true_context_id": "the_forest_island",
    "true_answer": "Tutup hari Jumat."
  },
  {
    "question": "Jam operasional The Forest Island?",
    "true_context_id": "the_forest_island",
    "true_answer": "Buka 8 AM dan tutup 5 PM (Sabtu-Kamis)."
  },
  {
    "question": "Di mana lokasi Taman Lokasana Asri?",
    "true_context_id": "taman_lokasana_asri",
    "true_answer": "Di Somagede, Banyumas."
  },
  {
    "question": "Fasilitas di Taman Lokasana Asri?",
    "true_context_id": "taman_lokasana_asri",
    "true_answer": "3 kolam renang dan taman."
  },
  {
    "question": "Berapa tiket Taman Lokasana Asri?",
    "true_context_id": "taman_lokasana_asri",
    "true_answer": "Rp10.000."
  },
  {
    "question": "Jam operasional Taman Lokasana Asri?",
    "true_context_id": "taman_lokasana_asri",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Menara Pandang Teratai?",
    "true_context_id": "menara_pandang_teratai",
    "true_answer": "Di Purwokerto, Banyumas."
  },
  {
    "question": "Apa keunikan Menara Pandang Teratai?",
    "true_context_id": "menara_pandang_teratai",
    "true_answer": "Ikon wisata berbentuk bunga teratai (pemandangan 360°)."
  },
  {
    "question": "Berapa tiket Menara Pandang Teratai (weekday)?",
    "true_context_id": "menara_pandang_teratai",
    "true_answer": "Rp20.000."
  },
  {
    "question": "Berapa tiket Menara Pandang Teratai (weekend)?",
    "true_context_id": "menara_pandang_teratai",
    "true_answer": "Rp25.000."
  },
  {
    "question": "Jam operasional Menara Pandang Teratai?",
    "true_context_id": "menara_pandang_teratai",
    "true_answer": "Buka 9 AM dan tutup 10 PM."
  },
  {
    "question": "Di mana lokasi Dreamland?",
    "true_context_id": "Dreamland",
    "true_answer": "Di Pancasan, Kec. Ajibarang."
  },
  {
    "question": "Wahana di Dreamland?",
    "true_context_id": "Dreamland",
    "true_answer": "Water boom, Kolam arus, Flying Fox, dan Taman reptil."
  },
  {
    "question": "Berapa tiket Dreamland (weekday)?",
    "true_context_id": "Dreamland",
    "true_answer": "Rp20.000."
  },
  {
    "question": "Berapa tiket Dreamland (weekend)?",
    "true_context_id": "Dreamland",
    "true_answer": "Rp25.000."
  },
  {
    "question": "Jam operasional Dreamland?",
    "true_context_id": "Dreamland",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Buken?",
    "true_context_id": "buken",
    "true_answer": "Di Banjarsari Kulon, Kec Sumbang."
  },
  {
    "question": "Fasilitas di Buken?",
    "true_context_id": "buken",
    "true_answer": "Kolam renang mata-air dan Taman satwa mini."
  },
  {
    "question": "Berapa tiket Buken (weekday)?",
    "true_context_id": "buken",
    "true_answer": "Rp10.000."
  },
  {
    "question": "Berapa tiket Buken (weekend)?",
    "true_context_id": "buken",
    "true_answer": "Rp15.000."
  },
  {
    "question": "Jam operasional Buken?",
    "true_context_id": "buken",
    "true_answer": "Buka 7.30 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Wisata Husada Kalibacin?",
    "true_context_id": "husada_kalibacin",
    "true_answer": "Di Kalibacin, Rawalo."
  },
  {
    "question": "Apa keunikan Wisata Husada Kalibacin?",
    "true_context_id": "husada_kalibacin",
    "true_answer": "Pemandian cagar budaya (sejak abad ke-17 M) untuk terapi kesehatan."
  },
  {
    "question": "Berapa tiket Wisata Husada Kalibacin?",
    "true_context_id": "husada_kalibacin",
    "true_answer": "Rp5.000."
  },
  {
    "question": "Jam operasional Wisata Husada Kalibacin?",
    "true_context_id": "husada_kalibacin",
    "true_answer": "Buka 7 AM dan tutup 4.30 PM."
  },
  {
    "question": "Di mana lokasi Prawita Garden?",
    "true_context_id": "prawita_garden",
    "true_answer": "Di Pegawulan Tengah, Darmakradenan, Kec. Ajibarang."
  },
  {
    "question": "Aktivitas di Prawita Garden?",
    "true_context_id": "prawita_garden",
    "true_answer": "Wisata edukasi agro (belajar memanen madu dan buah)."
  },
  {
    "question": "Berapa tiket Prawita Garden?",
    "true_context_id": "prawita_garden",
    "true_answer": "Tidak disebutkan."
  },
  {
    "question": "Jam operasional Prawita Garden?",
    "true_context_id": "prawita_garden",
    "true_answer": "Buka 7 AM dan tutup 9 PM."
  },
  {
    "question": "Di mana lokasi New Small Word?",
    "true_context_id": "new_small_word",
    "true_answer": "Di Ketenger, Baturraden."
  },
  {
    "question": "Aktivitas di New Small Word?",
    "true_context_id": "new_small_word",
    "true_answer": "Fotografi dan Melihat replika miniatur ikonik dunia."
  },
  {
    "question": "Berapa tiket New Small Word (weekday)?",
    "true_context_id": "new_small_word",
    "true_answer": "Rp15.000."
  },
  {
    "question": "Berapa tiket New Small Word (weekend)?",
    "true_context_id": "new_small_word",
    "true_answer": "Rp20.000."
  },
  {
    "question": "Jam operasional New Small Word?",
    "true_context_id": "new_small_word",
    "true_answer": "Buka 7 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi Wisata Pereng?",
    "true_context_id": "pereng",
    "true_answer": "Di Cilongok, Banyumas."
  },
  {
    "question": "Aktivitas edukasi di Wisata Pereng?",
    "true_context_id": "pereng",
    "true_answer": "Wisata edukasi pembuatan gula merah."
  },
  {
    "question": "Berapa tiket Wisata Pereng?",
    "true_context_id": "pereng",
    "true_answer": "Rp20.000."
  },
  {
    "question": "Jam operasional Wisata Pereng?",
    "true_context_id": "pereng",
    "true_answer": "Buka 8 AM dan tutup 5 PM."
  },
  {
    "question": "Di mana lokasi The Village?",
    "true_context_id": "the_village",
    "true_answer": "Di Rempoah, Baturraden, Banyumas."
  },
  {
    "question": "Apa tema The Village?",
    "true_context_id": "the_village",
    "true_answer": "Nuansa Eropa klasik (untuk swafoto)."
  },
  {
    "question": "Berapa tiket The Village?",
    "true_context_id": "the_village",
    "true_answer": "Rp25.000."
  },
  {
    "question": "Hari apa The Village tutup?",
    "true_context_id": "the_village",
    "true_answer": "Tutup hari Selasa."
  },
  {
    "question": "Jam operasional The Village?",
    "true_context_id": "the_village",
    "true_answer": "Buka 8 AM dan tutup 5 PM (kecuali Selasa)."
  },
  {
    "question": "Di mana lokasi Randu Bengkong?",
    "true_context_id": "randu_bengkong",
    "true_answer": "Di Sikapat, Kec Sumbang."
  },
  {
    "question": "Apa keunikan air di Randu Bengkong?",
    "true_context_id": "randu_bengkong",
    "true_answer": "Berasal dari mata air yang selalu mengalir (dingin dan bersih)."
  },
  {
    "question": "Berapa tiket dewasa Randu Bengkong?",
    "true_context_id": "randu_bengkong",
    "true_answer": "Rp10.000."
  },
  {
    "question": "Berapa tiket anak Randu Bengkong?",
    "true_context_id": "randu_bengkong",
    "true_answer": "Rp8.000."
  },
  {
    "question": "Jam operasional Randu Bengkong?",
    "true_context_id": "randu_bengkong",
    "true_answer": "Buka 8 AM dan tutup 4 PM."
  },
  {
    "question": "Di mana lokasi Lembu Benggolo?",
    "true_context_id": "lembu_benggolo_farm",
    "true_answer": "Di Kedunggede, Kec. Lumbir."
  },
  {
    "question": "Apa daya tarik Lembu Benggolo?",
    "true_context_id": "lembu_benggolo_farm",
    "true_answer": "Mini zoo, Resto saung, Rainbow slide, dan ATV."
  },
  {
    "question": "Berapa tiket masuk Lembu Benggolo?",
    "true_context_id": "lembu_benggolo_farm",
    "true_answer": "Rp5.000 (tidak termasuk wahana)."
  },
  {
    "question": "Jam operasional Lembu Benggolo?",
    "true_context_id": "lembu_benggolo_farm",
    "true_answer": "Buka 8 AM dan tutup 10 PM."
  },
  {
    "question": "Di mana lokasi Taman Botani?",
    "true_context_id": "taman_botani",
    "true_answer": "Di Karangmangu, Kec. Baturaden."
  },
  {
    "question": "Apa koleksi Taman Botani?",
    "true_context_id": "taman_botani",
    "true_answer": "Anggrek, kaktus, bonsai, dan bromelia."
  },
  {
    "question": "Berapa tiket Taman Botani (weekday)?",
    "true_context_id": "taman_botani",
    "true_answer": "Rp15.000."
  },
  {
    "question": "Berapa tiket Taman Botani (weekend)?",
    "true_context_id": "taman_botani",
    "true_answer": "Rp20.000."
  },
  {
    "question": "Jam operasional Taman Botani?",
    "true_context_id": "taman_botani",
    "true_answer": "Buka 9 AM dan tutup 10 PM."
  },
  {
    "question": "Di mana lokasi Arcelio Aquapark?",
    "true_context_id": "arcelio_aquapark",
    "true_answer": "Di Bancar Kembar, Purwokerto."
  },
  {
    "question": "Wahana di Arcelio Aquapark?",
    "true_context_id": "arcelio_aquapark",
    "true_answer": "Waterpark, Lazy river, dan Water playground."
  },
  {
    "question": "Berapa tiket Arcelio Aquapark (weekend)?",
    "true_context_id": "arcelio_aquapark",
    "true_answer": "Rp 20.000 (normal) atau Rp 30.000 (khusus berenang)."
  },
  {
    "question": "Jam operasional Arcelio Aquapark?",
    "true_context_id": "arcelio_aquapark",
    "true_answer": "Buka 6 AM dan tutup 6 PM."
  },
  {
    "question": "Di mana lokasi Goa Maria Kaliori?",
    "true_context_id": "goa_maria_kaliori",
    "true_answer": "Di Kaliori, Banyumas."
  },
  {
    "question": "Aktivitas di Goa Maria Kaliori?",
    "true_context_id": "goa_maria_kaliori",
    "true_answer": "Wisata religi, Ziarah, dan Retret."
  },
  {
    "question": "Berapa tiket Goa Maria Kaliori?",
    "true_context_id": "goa_maria_kaliori",
    "true_answer": "Tidak disebutkan."
  },
  {
    "question": "Jam operasional Goa Maria Kaliori?",
    "true_context_id": "goa_maria_kaliori",
    "true_answer": "24 Hours."
  }
]

    

# === 1. Inisialisasi List Hasil ===
retrieval_true = []
retrieval_pred = []
retrieval_precisions = []
generation_preds = []
generation_refs = []
latencies = []

print(f"\nMemulai evaluasi untuk {len(test_set)} data uji...")


for item in test_set:
    try: 
        question = item["question"]
        true_id = item["true_context_id"]
        true_ans = item["true_answer"]

        # --- A. Uji Retrieval (K=3) ---
        # Ganti 'retriever' dengan fungsi/objek retriever Anda
        retrieved_docs = retriever.get_relevant_documents(question)
        retrieved_ids = [doc.metadata.get('id', "NO_ID") for doc in retrieved_docs]
        
        is_hit = 1 if true_id in retrieved_ids else 0
        retrieval_true.append(1)
        retrieval_pred.append(is_hit)
        
        k = len(retrieved_docs)
        precision_query = (1 / k) if is_hit and k > 0 else 0.0
        retrieval_precisions.append(precision_query)
    
        # --- B. Uji Generasi & Latency ---
        t0 = time.perf_counter()
        
        response = rag_chain.invoke({"input": question})
        lat = time.perf_counter() - t0

        predicted_answer = (
            response.get("answer")
            or response.get("output_text")
            or response.get("result")
            or ""
        ).strip()

        if not predicted_answer:
            predicted_answer = "Maaf, aku belum menemukan informasi yang cocok."

        # Kumpulkan semua hasil untuk dihitung nanti
        generation_preds.append(predicted_answer)
        generation_refs.append(true_ans)
        latencies.append(lat)
        
    except Exception as e:
        print(f"\n--- ERROR FATAL PADA ITEM ---")
        print(f"Gagal memproses pertanyaan: {item.get('question', 'PERTANYAAN TIDAK DIKETAHUI')}")
        print(f"Error: {e}")
        print(f"Melanjutkan ke data berikutnya...\n")
        continue 
# === AKHIR DARI LOOP ===

print("Evaluasi selesai.")
print(f"Total data yang BERHASIL dievaluasi: {len(retrieval_true)} dari {len(test_set)} data.")

# =============================================================
# === D. KALKULASI & TAMPILKAN HASIL ===
# =============================================================

print("\n" + "="*60)
print("HASIL EVALUASI AKHIR")
print("="*60)

# A. Retrieval Performance 
if len(retrieval_true) > 0:
    r_recall = recall_score(retrieval_true, retrieval_pred, zero_division=0)
    r_precision = np.mean(retrieval_precisions) if retrieval_precisions else 0.0
    
    r_f1 = 0.0
    if (r_precision + r_recall) > 0:
        r_f1 = 2 * (r_precision * r_recall) / (r_precision + r_recall)
    
    print("\n=== Retrieval Performance ===")
    print(f"Precision : {r_precision:.4f}")
    print(f"Recall    : {r_recall:.4f}")
    print(f"F1-Score : {r_f1:.4f}")
else:
    print("\nTidak ada data retrieval yang berhasil dievaluasi!")

# B. Generation Quality (BLEU & BERTSCORE)
if generation_preds:
    print("\n=== Generation Quality (BLEU & BERTSCORE) ===")
    
    P_bert, R_bert, F1_bert = bert_scorer(
        generation_preds, 
        generation_refs, 
        lang='id', 
        verbose=False,
    )
    
    avg_bert_p = P_bert.mean().item()
    avg_bert_r = R_bert.mean().item()
    avg_bert_f1 = F1_bert.mean().item()
    
    print(f"BERTSCORE Precision : {avg_bert_p:.4f}")
    print(f"BERTSCORE Recall    : {avg_bert_r:.4f}")
    print(f"BERTSCORE F1        : {avg_bert_f1:.4f}")

    # 2. BLEU Score 
    refs_tokenized = [[ref.lower().split()] for ref in generation_refs]
    preds_tokenized = [pred.lower().split() for pred in generation_preds]
    
    smoothie = SmoothingFunction()
    
    bleu_2 = corpus_bleu(
        refs_tokenized, 
        preds_tokenized, 
        # Menggunakan BLEU-2 (0.5, 0.5) 
        weights=(0.5, 0.5, 0, 0), 
        # Menggunakan method3 yang lebih stabil untuk jawaban pendek
        smoothing_function=smoothie.method3 
    )
    
    print(f"\nCorpus BLEU-2 Score (Method 3) : {bleu_2:.4f}")
    
else:
    print("\nTidak ada data generasi yang berhasil dievaluasi!")

# --- C. Generation Performance (Latency) ---
if latencies:
    avg_latency = np.mean(latencies)
    min_latency = np.min(latencies)
    max_latency = np.max(latencies)
    
    print("\n=== Generation Performance (Latency) ===")
    print(f"Avg Response Time : {avg_latency:.3f} sec")
    print(f"Min Response Time : {min_latency:.3f} sec")
    print(f"Max Response Time : {max_latency:.3f} sec")
else:
    print("\n Tidak ada data latency yang berhasil dikumpulkan!")

print("\n" + "="*60)

## multi question